# DUAW recomender system

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
def display_all(df):
    with pd.option_context('display.max_rows',None,'display.max_columns', None):
        display(df)  

## API request Repository names and description
https://www.nylas.com/blog/use-python-requests-module-rest-apis/

In [18]:
import requests

In [19]:
url = "https://api.github.com/orgs/faketechlabsberlin/repos"
repos = requests.get(url)
repos_json = repos.json()
print(type(repos_json))

<class 'list'>


In [20]:
#Authentification for the get API
#requests.get('https://api.github.com/user', auth=HTTPBasicAuth('username', 'password'))

In [21]:
data_df = pd.DataFrame(repos_json)

### Similarity analysis

In [22]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
corpus = data_df.description
repo_names = data_df.name

##### 1) vectorization - Sentence Transformation using BERT  'all-mpnet-base-v2' 

**BERT Model 2: all-mpnet-base-v2** 
https://huggingface.co/sentence-transformers/all-mpnet-base-v2

In [38]:
model_2 = 'all-mpnet-base-v2' 

In [39]:
sen = corpus
model = SentenceTransformer(model_2)#''
sen_embeddings = model.encode(sen)
sen_embeddings.shape
#Embedding returns a dense matrix, where each row represents the vectorized text for one project, that can be further analysed

(12, 768)

##### 2)  Pairwise similarity analysis for all repositories

In [40]:
model_2_cosine_df =  pd.DataFrame(cosine_similarity(sen_embeddings))

##### 3) Conversion of results into list of results, sorted from most similar to least similar

In [41]:
#sorting of results from similarity analysis
def sorted_results_by_name(df = []):
    list_of_lists =[]
    for each in df.columns: # for each column in data frame
        list_series=[]
        current_series = df[each]
        sorted_series = current_series.sort_values(ascending=False)
       # return list with repositoryies
        
        for each in sorted_series.index:
            list_series.append(repo_names[each]) # append list 
        list_of_lists.append(list_series)

    return pd.DataFrame(list_of_lists)

In [42]:
results_df = sorted_results_by_name(df = model_2_cosine_df)
highest_results=results_df.iloc[:,0:4]

## export results into JSON file

In [43]:
import json

In [44]:
# creat a dictionary from Dataframe results
highest_results_dic = highest_results.set_index(0).T.to_dict('list')

In [45]:
# Convert Dictionary to JSON and export it to a JSON file
#highest_results_dic_json = json.dumps(highest_results_dic, sort_keys=True, indent=4)
with open('recomendations.json', 'w') as f:
    json.dump(highest_results_dic, f,  sort_keys=True, indent=4)